<a href="https://colab.research.google.com/github/agusnieto77/taller_pict_2019/blob/main/Colabs/ACEP_laboratorio_VIII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicio 8

In [ ]:
install.packages("ACEP")

In [ ]:
require(httr)
require(jsonlite)
require(purrr)
require(googledrive)

In [ ]:
drive_download(file = as_id("1xkQb3k93nBP2kVCQgl6I1TkuK7NEUSzw"), path = "api_gpt.rds")
drive_download(file = as_id("10JcWvdV2aFm22vfWPWkHJQ_9hiB4sNuz"), path = "osal_arg_tot_230510.rds")

In [11]:
# Datos
osal_arg_tot <- readRDS("/content/osal_arg_tot_230510.rds")[1:3,2:3]
vectro_osal <- paste0("Fecha: ", osal_arg_tot$fecha, ". ", osal_arg_tot$texto)
text_vector <- vectro_osal
text_vector

[1] "Fecha: 2000-01-12. La CTA y la CGT realizaron sendas movilizaciones en la Provincia de Corrientes en contra de la reducción de la planta del personal y del pago de sueldos con bonos anunciada por el Interventor Federal actualmente a cargo del gobierno de la Provincia."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
[2] "Fecha: 2000-01-24. Empleados municipales de la ciudad de Gral. Güemes, Salta, iniciaron el corte de la ruta nacional 34 en demanda de los sueldos atrasados desde hace 9 meses. Ex obreros del quebrado ingenio San Isidro se sumaron a la manifestación para reclamar el pago de indemnizaciones adeudadas."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
[3] "Fecha: 2000-01-26. Trabajadores estatales y del sindicato de Luz y Fuerza realizaron una marcha en la ciudad de Córdoba en oposición a la ley ómnibus enviada por el Gobierno Provincial al Parlamento local. Dicha ley prevee entre otras cuestiones, la privatización de EPEC (Empresa Provincial de Energía Eléctrica) y del Banco de Córdoba y determina duros parámetros de disciplina fiscal.  El Sindicato de Peones de Taxi realizo una marcha a Plaza de Mayo para entregar un petitorio en el ministerio del Interior reclamando más seguridad. 59  La policía salteña reprimió a los trabajadores que cortaban la ruta nacional 34 en la provincia de Salta. 48 detenidos, entre ellos dirigentes gremiales y un periodista, y un número no definido de lesionados fue el resultado del operativo"

In [12]:
# función ACEP
acep_gpt_tripletes <- function(text_vector, gpt_api) {
  triplets <- c()
  vector <- paste("Su tarea es generar un JSON que incluya solo 4 claves:",
                  "'fecha', 'sujeto', 'accion', 'objeto', 'lugar'.\n",
                  "La clave 'fecha' debe identificar el formato 'yyyy-mm-dd'",
                  "de la fecha de la acción de protesta descripta en el resumen.\n",
                  "La clave 'sujeto' debe identificar en un máximo de 5 palabras",
                  "quién realiza la acción de protesta descripta en el resumen.\n",
                  "La clave 'accion' debe identificar en un máximo de 3 palabras",
                  "el formato de la acción de protesta descripta en el resumen.\n",
                  "La clave 'objeto' debe identificar en un máximo de 6 palabras",
                  "contra quién o contra qué se realiza la acción de protesta",
                  "descripta en el resumen.\n",
                  "La clave 'lugar' debe identificar en un máximo de 4 palabras",
                  "la localización geográfica dónde se realiza la acción de protesta",
                  "descripta en el resumen. Si el resumen no informa sobre el lugar de",
                  "la protesta debe escribir 'sin datos'.\n",
                  "El resumen a analizar aparece a continuación",
                  "delimitado por '```': \n```", text_vector, "```", sep = " ")
  for (i in vector) {
    output <- httr::POST(
      url = "https://api.openai.com/v1/chat/completions", 
      httr::add_headers(Authorization = base::paste("Bearer", gpt_api)), 
      httr::content_type_json(),
      encode = "json",
      body = base::list(
        model = "gpt-3.5-turbo-0301",
        messages = base::list(
          base::list(
            role = "user",
            content = i
          )
        )
      )
    )
    
    triplets <- append(triplets, httr::content(output)$choices[[1]]$message$content)
  }
  return(triplets)
}

In [14]:
json_eventos <- acep_gpt_tripletes(text_vector = text_vector, gpt_api = readRDS("/content/api_gpt.rds"))
cat(paste0(json_eventos, collapse = "\n"))

{
 "fecha": "2000-01-12",
 "sujeto": "La CTA y la CGT",
 "accion": "realizaron movilizaciones",
 "objeto": "reducción de la planta del personal y pago de sueldos con bonos",
 "lugar": "Provincia de Corrientes"
}
{
  "fecha": "2000-01-24",
  "sujeto": "Empleados municipales y ex obreros",
  "accion": "Iniciaron corte",
  "objeto": "Sueldos e indemnizaciones adeudadas",
  "lugar": "Gral. Güemes, Salta"
}
{
  "fecha": "2000-01-26",
  "sujeto": "Trabajadores estatales y del sindicato de Luz y Fuerza",
  "accion": "realizaron una marcha",
  "objeto": "en oposición a la ley ómnibus enviada por el Gobierno Provincial al Parlamento local",
  "lugar": "ciudad de Córdoba"
}

In [15]:
(df <- purrr::map_df(json_eventos, ~jsonlite::fromJSON(.x), .id = "id"))

id,fecha,sujeto,accion,objeto,lugar
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2000-01-12,La CTA y la CGT,realizaron movilizaciones,reducción de la planta del personal y pago de sueldos con bonos,Provincia de Corrientes
2,2000-01-24,Empleados municipales y ex obreros,Iniciaron corte,Sueldos e indemnizaciones adeudadas,"Gral. Güemes, Salta"
3,2000-01-26,Trabajadores estatales y del sindicato de Luz y Fuerza,realizaron una marcha,en oposición a la ley ómnibus enviada por el Gobierno Provincial al Parlamento local,ciudad de Córdoba
